In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import warnings

warnings.filterwarnings('ignore')  # To clean up output


In [3]:

# Load dataset (classification example)
data = load_breast_cancer()
X, y = data.data, data.target

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:

# Define models and parameter grids
models_and_parameters = {
    'LinearRegression': {
        'model': LinearRegression(),
        'params': {}  # No hyperparameters to tune
    },
    'Ridge': {
        'model': Ridge(),
        'params': {
            'alpha': [0.1, 1.0, 10, 100],
            'solver': ['auto', 'svd', 'cholesky']
        }
    },
    'Lasso': {
        'model': Lasso(max_iter=10000),
        'params': {
            'alpha': [0.01, 0.1, 1, 10],
            'max_iter': [1000, 5000, 10000]
        }
    },
    'SVM': {
        'model': SVC(),
        'params': {
            'kernel': ['linear', 'rbf'],
            'C': [0.1, 1, 10],
            'gamma': ['scale', 'auto']
        }
    },
    'LogisticRegression': {
        'model': LogisticRegression(max_iter=5000),
        'params': {
            'C': [0.01, 0.1, 1, 10],
            'penalty': ['l2'],  # 'l1' requires solver='liblinear' or 'saga'
            'solver': ['lbfgs', 'saga']
        }
    },
    'DecisionTree': {
        'model': DecisionTreeClassifier(),
        'params': {
            'max_depth': [None, 5, 10, 20],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        }
    },
    'RandomForest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 5, 10],
            'min_samples_split': [2, 5],
            'min_samples_leaf': [1, 2]
        }
    }
}


In [5]:

# Run GridSearchCV for each model
for model_name, mp in models_and_parameters.items():
    print(f"\n--- {model_name} ---")
    
    # For some models, scaling is important (e.g., SVM, LogisticRegression)
    # So we can use a Pipeline for those models.
    if model_name in ['SVM', 'LogisticRegression', 'Ridge', 'Lasso', 'LinearRegression']:
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('model', mp['model'])
        ])
        param_grid = {}
        # Adjust param names to pipeline format
        for param, values in mp['params'].items():
            param_grid[f'model__{param}'] = values
        grid = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy' if model_name != 'LinearRegression' else 'r2')
    else:
        # For tree-based models scaling not necessary
        grid = GridSearchCV(mp['model'], mp['params'], cv=5, scoring='accuracy')
    
    # Fit grid search
    grid.fit(X_train, y_train)
    
    print("Best params:", grid.best_params_)
    print("Best CV score:", grid.best_score_)



--- LinearRegression ---
Best params: {}
Best CV score: 0.692497204942028

--- Ridge ---
Best params: {'model__alpha': 0.1, 'model__solver': 'auto'}
Best CV score: nan

--- Lasso ---
Best params: {'model__alpha': 0.01, 'model__max_iter': 1000}
Best CV score: nan

--- SVM ---
Best params: {'model__C': 0.1, 'model__gamma': 'scale', 'model__kernel': 'linear'}
Best CV score: 0.9736263736263737

--- LogisticRegression ---
Best params: {'model__C': 1, 'model__penalty': 'l2', 'model__solver': 'lbfgs'}
Best CV score: 0.9758241758241759

--- DecisionTree ---
Best params: {'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 5}
Best CV score: 0.9362637362637363

--- RandomForest ---
Best params: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
Best CV score: 0.9604395604395604
